In [2]:
### What is a Hash Function?
## The hash function can take any size of input and any type of input, it doesn't have to be readable text like this. -> Hash Function -> Fixed size binary!

## Properties of  a Hash Function
## 1. It is an one-way deterministic function
## 2. The output is dependent on all input bits
## 3. Output is uniformly distributed
## 4. "Impossible" (difficult) make a collision

## Use of Hash Functions
## 1. Digital signature
## 2. Shadow files (passwords)
## 3. HMAC
## 4. Make deterministic identifiers

## What is HMAC?
## HMAC (sometimes expanded as either keyed-hash message authentication code or hash-based message authentication code)
## is a specific type of message authentication code (MAC) involving a cryptographic hash function and a secret cryptographic key. 
## As with any MAC, it may be used to simultaneously verify both the data integrity and authenticity of a message.
## HMAC can provide authentication using a shared secret instead of using digital signatures with asymmetric cryptography. 
## It trades off the need for a complex public key infrastructure by delegating the key exchange to the communicating parties, 
## who are responsible for establishing and using a trusted channel to agree on the key prior to communication.


In [3]:
import hashlib

In [5]:
def modify(m):
  l = list(m)
  l[0] = l[0] ^ 1
  return bytes(l)

In [4]:
m = "This is the hash value message".encode()

sha256 = hashlib.sha256()
sha256.update(m)
d = sha256.digest()

print(d)

b"g\xf2\x05\x13\xd6\x88\xef\xf8WX\xe4\x7f9\x03\xdeJ\xbf&\x06\x8f\x94'\xf6\xf0\x0c\x0c\xd4\x7fu\x0b@\xfe"


In [6]:
m = modify(m)
print(m)

b'Uhis is the hash value message'


In [7]:
sha256 = hashlib.sha256()
sha256.update(m)
d = sha256.digest()

print(d)

b'\xeb\x88\x94hg\xaex\xdc\x16\xd0p\x14\x0bC\xfb\xe0\xa2\xcd3\x1d\x83\x14\xe4cg\xd7;\xab\x90S\xc6\x8b'


In [17]:
## How Digital Signatures works (uses hash functions) and implementation of it.
## Digital Signature
# These are Alice's RSA keys
# Public key (e,n): 5 170171
# Secret key (d) 9677
n = 170171
e = 5
d = 9677

# This is the message that Alice wants to sign and send to Bob
message = "Bob you are awesome".encode()

# Step 1: hash the message
sha256 = hashlib.sha256()
sha256.update(message)
h = sha256.digest()
h = int.from_bytes(h, "big") % n
#h = int.from_bytes(h, "big")
print("Hash value:", h)




Hash value: 30570


In [18]:
# Step 2: decrypt the hash value (use secret exponent)
sign = h**d % n

# Step 3: send messahe with signature to Bob
print(message, sign)

b'Bob you are awesome' 43738


In [19]:
# Bob verifying the signature
# Step 1: calculate the hash value of the message
sha256 = hashlib.sha256()
sha256.update(message)
h = sha256.digest()
h = int.from_bytes(h, "big") % n
#h = int.from_bytes(h, "big")
print("Hash value:", h)

# Step 2: Verify the signature
verification = sign**e % n
print("Verification value:", verification)

Hash value: 30570
Verification value: 30570


In [20]:
## Eve trying to modify a signed message - will Bob figure it out?
# This is Eve being evil and modifies the message
message = modify(message)
print(message)

b'Cob you are awesome'


In [26]:
## How Passwords are Verified - with real example from Mac
# ShadowFile.xml
# PBKDF2: PBKDF1 and PBKDF2 (Password-Based Key Derivation Function 1 and 2) 
# are key derivation functions with a sliding computational cost, used to reduce 
# vulnerabilities of brute-force attacks

import hashlib
import base64

iterations = 45454
salt = base64.b64decode("6VuJKkHVTdDelbNMPBxzw7INW2NkYlR/LoW4OL7kVAI=".encode())
#SALTED-SHA512-PBKDF2

password = "password".encode()
print(type(password))

value = hashlib.pbkdf2_hmac("sha512",password,salt,iterations,dklen=126)
print(base64.b64encode(value))

<class 'bytes'>
b'1meJW2W6Zugz3rKm/n0yysV+5kvTccA7EuGejmyIX8X/MFoPxmmbCf3BE62h6wGyWk/TXR7pvXKgjrWjZyI+Fc3aKfv1LNQ0/Qrod3lVJcWd9V6Ygt+MYU8Eptv3uwDcYf6Z5UuF+Hg67rpoDAWhJrC1PEfL3vcN7IoBqC5N'


In [32]:
## Why use Salt and shown by implementation in Python
iterations = 45454
salt = base64.b64decode("6VuJKkHVTdDelbNMPBxzw7INW2NkYlR/LoW4OL7kVAI=".encode())
#salt = "".encode()
validation = "SALTED-SHA512-PBKDF2"

password = "password".encode()


value = hashlib.pbkdf2_hmac("sha512",password,salt,iterations,dklen=126)
entropy = base64.b64encode(value)
print("Alice", validation, salt, iterations, entropy)

salt = "666".encode()
password = "password".encode()
value = hashlib.pbkdf2_hmac("sha512",password,salt,iterations,dklen=126)
entropy = base64.b64encode(value)
print("Bob", validation, salt, iterations, entropy)

Alice SALTED-SHA512-PBKDF2 b'\xe9[\x89*A\xd5M\xd0\xde\x95\xb3L<\x1cs\xc3\xb2\r[cdbT\x7f.\x85\xb88\xbe\xe4T\x02' 45454 b'1meJW2W6Zugz3rKm/n0yysV+5kvTccA7EuGejmyIX8X/MFoPxmmbCf3BE62h6wGyWk/TXR7pvXKgjrWjZyI+Fc3aKfv1LNQ0/Qrod3lVJcWd9V6Ygt+MYU8Eptv3uwDcYf6Z5UuF+Hg67rpoDAWhJrC1PEfL3vcN7IoBqC5N'
Bob SALTED-SHA512-PBKDF2 b'666' 45454 b'3DFPsnKGVVlRLNvpChoIPObu3sJy9TkpmagjOVHioSS59Rx1/rzbXNteXqG2IRsCPFFOvJa1sPbMpeYCLpXBnuT0evUL6DGYrJJNY1U3pnl18XvsMp47jvxBg+iIc95CPzg9NtOI4SKOxrCin9DlQwnJErCURwU2OgUjn207'


In [33]:
value = hashlib.pbkdf2_hmac("sha512",password,salt,iterations,dklen=126)
entropy = base64.b64encode(value)
print(entropy)

b'3DFPsnKGVVlRLNvpChoIPObu3sJy9TkpmagjOVHioSS59Rx1/rzbXNteXqG2IRsCPFFOvJa1sPbMpeYCLpXBnuT0evUL6DGYrJJNY1U3pnl18XvsMp47jvxBg+iIc95CPzg9NtOI4SKOxrCin9DlQwnJErCURwU2OgUjn207'


In [40]:
## Why iterate over hash function and demonstrated by implementation in Python

def guess_password(salt, iterations, entropy):
  alphabet = "abcdefghijklmnopqrstuvwxyz"
  for c1 in alphabet:
    for c2 in alphabet:
      password = str.encode(c1+c2)
      value = hashlib.pbkdf2_hmac("sha512",password,salt,iterations,dklen=128)
      if value == entropy:
        return password
  return "".encode()

iterations = 45454 # 1 # 45454
salt = base64.b64decode("6VuJKkHVTdDelbNMPBxzw7INW2NkYlR/LoW4OL7kVAI=".encode())
validation = "SALTED-SHA512-PBKDF2"
entropy = "3DFPsnKGVVlRLNvpChoIPObu3sJy9TkpmagjOVHioSS59Rx1/rzbXNteXqG2IRsCPFFOvJa1sPbMpeYCLpXBnuT0evUL6DGYrJJNY1U3pnl18XvsMp47jvxBg+iIc95CPzg9NtOI4SKOxrCin9DlQwnJErCURwU2OgUjn207"

password = "??".encode()

password = guess_password(salt, iterations, entropy)
print(password)
value = base64.b64encode(hashlib.pbkdf2_hmac("sha512", password, salt, iterations, dklen=128))
print(value)
print(entropy)

b''
b'eC5eeAhzcO/Og6bueVy4+v0I611CBz2rCPziTbsCiNVR+qulpqM5DYzn1HG8ox2jgwmmnVvdFe2ss9qfmxa0uc4+Q+nsHxaN4l2mfTR0GuDDEOSsko8COkw8UQMfzjcky1WdHM+/KJmKthUJBkEHOxpmws/SlzMPktmt8SAZ0Vo='
3DFPsnKGVVlRLNvpChoIPObu3sJy9TkpmagjOVHioSS59Rx1/rzbXNteXqG2IRsCPFFOvJa1sPbMpeYCLpXBnuT0evUL6DGYrJJNY1U3pnl18XvsMp47jvxBg+iIc95CPzg9NtOI4SKOxrCin9DlQwnJErCURwU2OgUjn207
